In [44]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [26]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [32]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.276s

OK


In [33]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [63]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

NotImplementedError: 

In [64]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'cponce7@gmail.com volkl200822366'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mrvpJNvSasgjUQPVnBsCeqJqSyeDKHxtjE


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [52]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.223s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [82]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
passphrase = b'cponce7@gmail.com volkl200822366'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)

# change address should be the address generated from Chapter 4
change_address = private_key.point.address(testnet=True)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx = bytes.fromhex('699ed8613af82e9cdb5da8a2aad6dff341819f7919e4b401164e89cf224c3e0e')
prev_index = 1

# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
tx_in = TxIn(prev_tx, prev_index)

# target amount should be 60% of the output amount
previous_transaction = tx_in.fetch_tx(testnet=True)
utxo_amount = previous_transaction.tx_outs[prev_index].amount
target_amount = int(utxo_amount*0.6)
#print(target_amount)

# set the fee to some reasonable amount
fee = 10000

# change amount = amount from the prev tx - target amount - fee
change_amount = utxo_amount - target_amount - fee
#print(change_amount)

# create a transaction output for the target amount and address
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(target_amount, target_script)

# create a transaction output for the change amount and address
change_h160 = decode_base58(change_address)
change_script = p2pkh_script(change_h160)
change_output = TxOut(change_amount, change_script)

# create the transaction object
tx = Tx(1, [tx_in], [target_output, change_output], 0, testnet=True)

# sign the one input in the transaction object using the private key
tx.sign_input(0, private_key)

# print the transaction's serialization in hex
print(tx.serialize().hex())

# broadcast at http://testnet.blockchain.info/pushtx
print()
print(tx)

01000000010e3e4c22cf894e1601b4e419799f8141f3dfd6aaa2a85ddb9c2ef83a61d89e69010000006b483045022100edb9f05f5233e366a26d6b2d90a80e7ce92094858231642ea4c19e934fce165502202aedb25cbf9da0f75a396bfb01be2ca609d859155e7ff302611411beb3c3ac340121028c0088dfc507ec5e0b8af48310b0c12cd29a55a97ed916d5e4849d1b1d1ee90dffffffff0260ea0000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac30750000000000001976a9147d2cb17d131aa5568867749c6cd252916ca57dee88ac00000000

tx: aceb898adbd033fadc881f7d4d8d555506241bdb08942f2b7be0227a752830a1
version: 1
tx_ins:
699ed8613af82e9cdb5da8a2aad6dff341819f7919e4b401164e89cf224c3e0e:1
tx_outs:
60000:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
30000:OP_DUP OP_HASH160 7d2cb17d131aa5568867749c6cd252916ca57dee OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [87]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mhHMWwEyyrW2Dw7cTbF627uLnFnqUj3EVe' #This is the target address I created, it's in chapter 4 notebook

# get the private key from the exercise in Chapter 4
passphrase = b'Input 2 cponce7@gmail.com volkl200822366'
secret = little_endian_to_int(hash256(passphrase))
private_key_input_faucet = PrivateKey(secret)

passphrase = b'cponce7@gmail.com volkl200822366'
secret = little_endian_to_int(hash256(passphrase))
private_key_input_ex4 = PrivateKey(secret)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx_input_faucet = bytes.fromhex('f4588d95818710d819105710a2234620d87d34ab4d15147b50893b7ead40981f')
prev_index_input_faucet = 1

# create the first transaction input with the default ScriptSig and
# sequence
tx_in_faucet = TxIn(prev_tx_input_faucet, prev_index_input_faucet)

# get the prev_tx and prev_index from the transaction in Exercise 4
prev_tx_input_ex4 = bytes.fromhex('aceb898adbd033fadc881f7d4d8d555506241bdb08942f2b7be0227a752830a1')
prev_index_input_ex4 = 1

# create the second transaction input with the default ScriptSig and
# sequence
tx_in_ex4 = TxIn(prev_tx_input_ex4, prev_index_input_ex4)

# set the fee to some reasonable amount
fee = 500

# target amount should be the sum of the inputs - fee
target_amount = tx_in_faucet.fetch_tx(testnet=True).tx_outs[prev_index_input_faucet].amount + tx_in_ex4.fetch_tx(testnet=True).tx_outs[prev_index_input_faucet].amount - fee

# create a transaction output for the amount and address
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)

tx_out = TxOut(target_amount, target_script)

# creating the tx object because this wasn't included in the instructions and I'm pretty sure it's needed
tx = Tx(1, [tx_in_faucet, tx_in_ex4], [tx_out], 0, testnet=True)

# sign the first input using the private key
tx.sign_input(0, private_key_input_faucet)

# sign the second input using the private key 
tx.sign_input(1, private_key_input_ex4)

# print the transaction's serialization in hex
print(tx.serialize().hex())
print()
print(tx)

# broadcast at http://testnet.blockchain.info/pushtx

01000000021f9840ad7e3b89507b14154dab347dd8204623a210571019d8108781958d58f4010000006a4730440220213fd7303ba79fe75007321c3e2576c4b415822b000f092c9974f773b656545e0220079bad50ee3be0d352c58b5b5e34f3e3b90458d6866222a719ff7ce0ed2389fe0121026a55176caab9cce97b79c55a4e236a34a7ff99d0bfc90c2b4b2dd5391edab734ffffffffa13028757a22e07b2b2f9408db1b240655558d4d7d1f88dcfa33d0db8a89ebac010000006a4730440220167e267b461623c41d6c5aef8fa386c5c98eb152dd43bd2274e2caf3ba7bf56f02203108dacffc9a5ce150a1171d3d986172fc2c7c643025eb8ba4518afca782cc6b0121028c0088dfc507ec5e0b8af48310b0c12cd29a55a97ed916d5e4849d1b1d1ee90dffffffff01dcf90100000000001976a914135dd01cb563b3b70c7f760eeaccf45d03b8797188ac00000000

tx: 4ecd3ba1b71724641ef834c4da86aba55e7621c968deab34c9cc21401796450c
version: 1
tx_ins:
f4588d95818710d819105710a2234620d87d34ab4d15147b50893b7ead40981f:1
aceb898adbd033fadc881f7d4d8d555506241bdb08942f2b7be0227a752830a1:1
tx_outs:
129500:OP_DUP OP_HASH160 135dd01cb563b3b70c7f760eeaccf45d03b87971 OP_EQUALVERIFY OP_CHECKSI